In [1]:
from math import pi
import numpy as np
import matplotlib.pyplot as plt
import wavio
from scipy import fftpack, signal
import IPython

In [2]:
def taps_convolver(sig, win, mode):
    u = np.zeros(len(win))
    M = len(win)
    
    if mode == 'full':
        # we will add (M-1) zeros at the end of input data
        # assume the last data is "C"
        # the last dot mutiply will look like this
        # [0,0,0,0,0,0,0,0,...., C] .* w[n0,n1,n2,n3,n4...]
        ITER = len(sig)+M-1
        y = np.zeros(len(sig)+M-1)
        sig = np.append(sig, np.zeros(M-1))
    elif mode == 'same':
        # the input length and output length is same
        ITER = len(sig)
        y = np.zeros(ITER)
    
    # fir taps processing
    for n in range(ITER):
        u[1:M] = u[0:M-1]
        u[0] = sig[n]
        y[n] = np.dot(win.T, u)
    return y

In [3]:
sig = np.array([6, -3, 13, 4, -5, 6, 7, -8, 1, 10, -1, -17, 3, 7, 1, 2])
# sig = np.array([6, -3, 13, 4, -5, 6, 7, -8])
win = np.array([1, -1, 1, -0.5, -1.5, 0, 1, 1])

In [4]:
# Compare signal.lfilter() and current implementation taps_convolver('full')
# the result is same!
a = signal.lfilter(win, 1, sig)
b = taps_convolver(sig, win, 'same')
print(a)
print(b)
print(a == b)

[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5
  25.5 -28.5   0.   43. ]
[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5
  25.5 -28.5   0.   43. ]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [5]:
# The output is the full discrete linear convolution of the inputs. (Default)
print(signal.convolve(sig, win, mode='full') / sum(win))

# the output is the same size as in1, centered with respect to the ‘full’ output.
print(signal.convolve(sig, win, mode='same') / sum(win))

# The output consists only of those elements that do not rely on the zero-padding. 
# In ‘valid’ mode, either in1 or in2 must be at least as large as the other in every dimension.
print(signal.convolve(sig, win, mode='valid') / sum(win)) 

[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5
  25.5 -28.5   0.   43.    0.  -27.  -16.5   7.    8.    3.    2. ]
[-15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5  25.5 -28.5   0.
  43.    0.  -27.  -16.5]
[ -9.5  30.5   5.5 -17.5   6.5  25.5 -28.5   0.   43. ]


In [6]:
# Compare signal.convolve('full') and current implementation taps_convolver('full')
# the result is same!
a = signal.convolve(sig, win, mode='full') / sum(win)
b = taps_convolver(sig, win, 'full')
print(a)
print(b)
print(a == b)

[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5
  25.5 -28.5   0.   43.    0.  -27.  -16.5   7.    8.    3.    2. ]
[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5
  25.5 -28.5   0.   43.    0.  -27.  -16.5   7.    8.    3.    2. ]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True]


In [7]:
# if we use fft method which means the signal and win were process in frequency domain
# so the result of two product is a little bit different
a = signal.convolve(sig, win, mode='full', method='fft') / sum(win)
b = taps_convolver(sig, win, 'full')
print(a)
print(b)
print(a == b)

[ 6.00000000e+00 -9.00000000e+00  2.20000000e+01 -1.50000000e+01
 -3.50000000e+00  1.30000000e+01 -1.95000000e+01 -9.50000000e+00
  3.05000000e+01  5.50000000e+00 -1.75000000e+01  6.50000000e+00
  2.55000000e+01 -2.85000000e+01  1.77635684e-15  4.30000000e+01
  0.00000000e+00 -2.70000000e+01 -1.65000000e+01  7.00000000e+00
  8.00000000e+00  3.00000000e+00  2.00000000e+00]
[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5
  25.5 -28.5   0.   43.    0.  -27.  -16.5   7.    8.    3.    2. ]
[False False False False  True False False False  True  True False False
 False  True False  True  True False False  True False False False]


In [8]:
# ifft(fft(sig) * fft(win))
#
# the result should match the convolution result(time domain)
# actually the full size of convolution(u[n]) is len(sig)+len(win)-1 
# we ignore the last dot multiply because it is 0.
#
# if we're going to do the convolution with fft approach
# we should double the size of fft first, so we could get len(sig)+len(win) number of fft product
# (and ignore the last element because it's 0)

# to make sure the size of ifft product will align the convolution result
sig_f = np.fft.fft(sig, (len(sig) + len(win)))

# to make sure the size of ifft product will align the convolution result
win_f = np.fft.fft(win, (len(sig) + len(win)))

# [a0b0,a1b1,a2b2,a3b3] = [a0,a1,a2,a3] * [b0,b1,b2,b3]
raw = np.fft.ifft(sig_f * win_f)

# we can ignore imaginary part, because it's all 0 :)
result = np.real(raw)

# disable science notation
np.set_printoptions(suppress=True)

# ignore the last element
print(result[:-1])
print(result[:-1] == a)

[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5  30.5   5.5 -17.5   6.5
  25.5 -28.5  -0.   43.    0.  -27.  -16.5   7.    8.    3.    2. ]
[ True False False False False False False  True  True  True False False
 False False False  True False False False False False False False]


In [9]:
res = taps_convolver(sig, win, 'full')
quotient, remainder = signal.deconvolve(res, sig)
print(quotient)
print(quotient == win)
quotient, remainder = signal.deconvolve(res, win)
print(quotient)
print(quotient == sig)

[ 1.  -1.   1.  -0.5 -1.5 -0.   1.   1. ]
[ True  True  True  True  True False False False]
[  6.  -3.  13.   4.  -5.   6.   7.  -8.   1.  10.  -1. -17.   3.   7.
   1.   2.]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


## OverLap-and-Add (OLA)

In [10]:
N = len(win)
m = len(sig)
M = m//2

block_1 = sig[:M]
block_2 = sig[M:]

res1 = taps_convolver(block_1, win, 'full')
res2 = taps_convolver(block_2, win, 'full')

output1 = res1[:M]
print(output1)

buffer = np.zeros(M)
buffer[:N-1] = res1[M:]
output2 = buffer + res2[:M]
print(output2)

[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5]
[ 30.5   5.5 -17.5   6.5  25.5 -28.5   0.   43. ]


another way to process overlap and add, in frequency domain

In [11]:
block_1 = sig[:M]
block_2 = sig[M:]

K = M + N - 1

win_padded = np.append(win, np.zeros(M-1))
block1_padded = np.append(block_1, np.zeros(N-1))
conv1 = np.fft.ifft(np.fft.fft(win_padded) * np.fft.fft(block1_padded))
output1 = conv1[:M].real
print(output1)

win_padded = np.append(win, np.zeros(M-1))
block2_padded = np.append(block_2, np.zeros(N-1))
conv2 = np.fft.ifft(np.fft.fft(win_padded) * np.fft.fft(block2_padded))

buffer = np.zeros(M)
buffer[:N-1] = conv1[M:].real
output2 = buffer + conv2[:M].real
print(output2)

[  6.   -9.   22.  -15.   -3.5  13.  -19.5  -9.5]
[ 30.5   5.5 -17.5   6.5  25.5 -28.5   0.   43. ]
